In [6]:
import pandas as pd

# Чтение файла (замените путь на реальный)
file_path = "VisitsStream.tsv"

# Просмотр заголовков таблицы
with open(file_path, "r") as f:
    headers = f.readline().strip()
print("Заголовки таблицы:", headers)

# Чтение данных с точными именами столбцов
data = pd.read_csv(file_path, sep="\t", header=0, names=["UserID", "IPID", "AdID", "ViewDate"])

# Преобразование 'ViewDate' в формат datetime
data["ViewDate"] = pd.to_datetime(data["ViewDate"])

# Проверка первых строк
data.head()


Заголовки таблицы: UserID	IPID	AdID	ViewDate


,UserID,IPID,AdID,ViewDate
0,59703,1259356,469877,2015-04-25
1,154389,1846749,27252551,2015-04-25
2,218628,2108380,31685325,2015-04-25
3,231535,837110,18827716,2015-04-25
4,282306,1654210,29363673,2015-04-25


In [11]:

# Функция mapper
def mapper(data):
    for index, row in data.iterrows():
        user_id = row["User ID"]
        ip_id = row["IPID"]
        ad_id = row["AdID"]
        view_date = row["ViewDate"]
        date_key = view_date.strftime('%Y-%m-%d')  # Преобразуем дату в формат 'YYYY-MM-DD'

        # Возвращаем данные в формате, удобном для reducer
        yield f"{user_id}\t{ip_id}\t{date_key}\t1"  # Добавляем 1 для подсчета

# Функция reducer
def reducer(mapped_data):
    current_key = None
    current_count = 0
    results = []

    for line in mapped_data:
        user_id, ip_id, view_date, count = line.strip().split('\t')
        count = int(count)

        key = f"{user_id}\t{ip_id}\t{view_date}"

        if current_key == key:
            current_count += count
        else:
            if current_key is not None:
                results.append(f"{current_key}\t{current_count}")
            current_key = key
            current_count = count

    if current_key is not None:
        results.append(f"{current_key}\t{current_count}")

    return results

# Запуск mapper и reducer
mapped_data = list(mapper(data))
reduced_results = reducer(mapped_data)

# Вывод результатов
for result in reduced_results:
    print(result)

MemoryError: 

In [ ]:
import pandas as pd
from collections import defaultdict

# Функция mapper
def mapper(data):
    for index, row in data.iterrows():
        user_id = row["User ID"]
        yield user_id, 1  # Возвращаем user_id и 1 для подсчета

# Функция reducer
def reducer(mapped_data):
    user_counts = defaultdict(int)

    for user_id, count in mapped_data:
        user_counts[user_id] += count

    # Сортируем пользователей по количеству посещений
    sorted_users = sorted(user_counts.items(), key=lambda x: x[1], reverse=True)

    # Возвращаем топ-10 пользователей
    return sorted_users[:10]

# Запуск mapper и reducer
mapped_data = list(mapper(data))
top_10_users = reducer(mapped_data)

# Вывод результатов
for user_id, count in top_10_users:
    print(f"{user_id}\t{count}")


In [ ]:
# Константа для максимального интервала в секундах (15 минут)
MAX_SESSION_INTERVAL = 15 * 60

def calculate_longest_sessions(data):
    # Группируем по UserID
    user_sessions = {}
    for user_id, group in data.groupby("UserID"):
        group = group.sort_values("ViewDate")
        longest_session = 0
        session_start = group["ViewDate"].iloc[0]
        
        for i in range(1, len(group)):
            diff = (group["ViewDate"].iloc[i] - group["ViewDate"].iloc[i - 1]).total_seconds()
            if diff > MAX_SESSION_INTERVAL:  # Новая сессия
                session_length = (group["ViewDate"].iloc[i - 1] - session_start).total_seconds()
                longest_session = max(longest_session, session_length)
                session_start = group["ViewDate"].iloc[i]
        
        # Проверяем последнюю сессию
        session_length = (group["ViewDate"].iloc[-1] - session_start).total_seconds()
        longest_session = max(longest_session, session_length)
        
        user_sessions[user_id] = longest_session
    
    return user_sessions

# Вычисление длинных сессий
user_sessions = calculate_longest_sessions(data)


In [2]:
# надо попробовать запустить эту

import dask.dataframe as dd

# Чтение данных с использованием Dask
ddf = dd.read_csv("VisitsStream.tsv", sep="\t", names=["UserID", "IPID", "AdID", "ViewDate"], header=0)
ddf["ViewDate"] = dd.to_datetime(ddf["ViewDate"])

# Константа для максимального интервала в секундах (15 минут)
MAX_SESSION_INTERVAL = 15 * 60

# Функция для вычисления длинных сессий
def calculate_sessions(group):
    group = group.sort_values("ViewDate")
    session_start = None
    longest_session = 0
    for i in range(len(group)):
        current_time = group["ViewDate"].iloc[i]
        if session_start is None or (current_time - session_start).total_seconds() > MAX_SESSION_INTERVAL:
            session_start = current_time
        else:
            session_length = (current_time - session_start).total_seconds()
            longest_session = max(longest_session, session_length)
    return longest_session

# Группировка и расчет
results = ddf.groupby("UserID").apply(calculate_sessions, meta=('x', 'float')).compute()
top_10_users = results.nlargest(10).compute()

# Вывод результата
top_10_users.reset_index().rename(columns={0: "LongestSessionSeconds"})

C:\Anaconda3\Lib\site-packages\dask_expr\_expr.py:1455: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return get_meta_library(args[0]).to_datetime(*args, **kwargs)


KeyboardInterrupt: 

In [35]:
from datetime import datetime
import heapq

def mapper(line):
    # Разбираем строку из TSV
    user_id, timestamp = line.strip().split('\t')
    timestamp = datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%S")
    
    # Выводим пару (user_id, timestamp)
    return (user_id, timestamp)

def reducer(user_id, timestamps):
    # Сортируем timestamps для каждого пользователя
    timestamps.sort()
    
    longest_session = 0
    session_start = timestamps[0]
    session_duration = 0
    
    for i in range(1, len(timestamps)):
        current_time = timestamps[i]
        prev_time = timestamps[i - 1]
        
        # Если разница больше 15 минут, заканчиваем старую сессию
        if (current_time - prev_time).seconds > 15 * 60:
            longest_session = max(longest_session, session_duration)
            session_start = current_time
            session_duration = 0
        
        # Обновляем продолжительность сессии
        session_duration = (current_time - session_start).seconds
    
    # Не забываем проверить последнюю сессию
    longest_session = max(longest_session, session_duration)
    
    return (user_id, longest_session)

In [ ]:

# Преобразуем временные метки в формат timestamp
data = data.withColumn("timestamp", unix_timestamp(col("timestamp"), "yyyy-MM-dd HH:mm:ss").cast("timestamp"))

# Сортируем данные по пользователям и времени
data = data.orderBy("user_id", "timestamp")

# Создаем окно для вычисления разницы во времени между соседними строками
window_spec = Window.partitionBy("user_id").orderBy("timestamp")

# Вычисляем разницу во времени между текущей и предыдущей записью
data = data.withColumn("prev_timestamp", lag("timestamp").over(window_spec))
data = data.withColumn("time_diff", (col("timestamp").cast("long") - col("prev_timestamp").cast("long")))

# Определяем начало и конец сессий
data = data.withColumn("session_start", F.when(col("time_diff").isNull() | (col("time_diff") > 900), col("timestamp")).otherwise(None))

# Заполняем значения session_start вперед по пользователям
data = data.withColumn("session_start", F.last("session_start", ignorenulls=True).over(window_spec))

# Вычисляем продолжительность сессии
data = data.withColumn("session_duration", (col("timestamp").cast("long") - col("session_start").cast("long")).when(col("session_start").isNotNull(), col("session_duration")).otherwise(0))

# Группируем данные по пользователям и находим максимальную продолжительность сессии
result = data.groupBy("user_id").agg(F.max("session_duration").alias("max_session_duration"))

# Сортируем и выбираем топ 10 пользователей
top_users = result.orderBy(col("max_session_duration").desc()).limit(10)

# Показать результаты
top_users.show()

# Завершаем сессию Spark
spark.stop()
